# Introduzione a Keras

Keras è una libreria di alto livello che consente di costruire e sperimentare con modelli di Deep Learning in maniera flessibile.
E' integrata con TensorFlow, che fornisce un supporto più di "basso livello". Documentazione, tutorial ed esempi sono nel sito web di TF.

Documentazione API (andate su `tf.keras`): https://www.tensorflow.org/versions

Guide Keras (per componenti API specifici): https://www.tensorflow.org/guide/keras

Tutorials (coprono esempi di utilizzo base): https://www.tensorflow.org/tutorials/keras

In [ ]:
import tensorflow as tf
from tensorflow import keras as K
import numpy as np
import seaborn as sns
import pandas as pd
sns.set_theme()

In [ ]:
K.__version__

# Primo training end-to-end con Keras

Cominciamo dal caricare il nostro dataset di immagini.

In [ ]:
import tensorflow.keras.datasets as kds
from PIL import Image

In [ ]:
(train_X, train_y), (test_X, test_y) = kds.mnist.load_data(path='ds')

In [ ]:
type(train_X)

In [ ]:
train_X.shape, train_y.shape, test_X.shape, test_y.shape

In [ ]:
train_X[0], train_y[0]

In [ ]:
print(f"Label is {train_y[0]}")
Image.fromarray(train_X[0])

In [ ]:
train_X.dtype, train_y.dtype

## Keras Pipeline

La prima cosa da fare con Keras è costruire la nostra "__scatola__" che ci permetterà di processare i dati da input ad output.

Vediamo il primo modo di costruirla, che è tramite la __Sequential API__

In [ ]:
model = K.Sequential()

Una pipeline è composta da **layers**.

__Un layer è una funzione che, dato un input, restituisce un output che è il risultato della "trasformazione" dell'input__. Solitamente (ma, come vedremo fra un attimo, non necessariamente), questo avviene rispetto a parametri adattivi.

Un modello si può costruire componendo molti layer, e mette a disposizione anche interfacce per funzionalità più complesse come il training, l'inferenza, ecc...

Dentro la Sequential API, possiamo inserire quanti layer vogliamo semplicemente utilizzando `model.add(new_layer)`. __Tutti i layer verranno eseguiti nell'ordine in cui li abbiamo inseriti__.



### Flatten Layer

Questo layer ci permette di appiattire i tensori, portandoli da un formato 28x28 a un vettore di lunghezza 784.

In [ ]:
model.add(K.layers.Flatten())

### Rescaling Layer

Questo layer riscala i dati applicando la formula `x = x * scale`. La motivazione dietro il rescaling è che vogliamo evitare che i neuroni si **saturino**.

In [ ]:
help(K.layers.Rescaling.__init__)

In [ ]:
model.add(K.layers.Rescaling(scale=1./255))

### Dense Layer

Questo layer ci permette di implementare qualsiasi funzione che prevede:

1. Una funzione lineare del tipo `y = Wx + b`;
2. Successivamente, una __funzione di attivazione__, che applica la nonlinearità della rete neurale.

In [ ]:
help(K.layers.Dense.__init__)

In [ ]:
model.add(K.layers.Dense(1000, activation='relu')) # E la dimensione di input?
model.add(K.layers.Dense(200, activation='relu'))
model.add(K.layers.Dense(10, activation='softmax')) 

In [ ]:
l2_reg = K.regularizers.l2(l2=0.5)

### Compiliamo la pipeline con un ottimizzatore

La funzione di compilazione è quella che ci permette di prendere tutti questi pezzetti che stiamo mettendo dentro la Sequential API, e "incastonarli" dentro un grafo computazionale.

Sarà Keras stesso, a questo punto, a curarsi di quali sono le parti che hanno dei parametri adattivi (che quindi hanno bisogno di gradiente) e quali no, rispetto al modo in cui sono definiti i layers.

In [ ]:
help(model.compile)

In [ ]:
model.compile(
    optimizer=K.optimizers.Adam(learning_rate=0.1),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

La __regolarizzazione__ è uno strumento fondamentale per ridurre la probabilità di __overfitting__ del nostro modello.

Generalmente, valori molto grandi dei pesi di un modello sono associati all'overfitting. Di conseguenza, aggiungendo un termine di penalizzazione alla loss è possibile indurre i pesi del modello ad avere valori più piccoli.
A seconda del tipo di penalizzazione, possiamo avere L1 (valore assoluto) o L2 (norma quadratica).

### Alleniamo il modello

Per allenare un modello dentro Keras è sufficiente chiamare il metodo `.fit`.

In [ ]:
help(model.fit)

In [ ]:
history = model.fit(
    x=train_X,
    y=train_y,
    epochs=500,
    batch_size=train_X.shape[0]
)

In [ ]:
model.summary()

Lla funzione `.fit` restituisce uno storico di ciò che è accaduto durante il training in termini di metriche.

In [ ]:
list(history.history)

Questo è molto utile per poter visualizzare il comportamento del modello dopo il training.

In [ ]:
ax = sns.lineplot(x=history.epoch, y=history.history['loss'])
ax.set(xlabel='epoch', ylabel='loss')

In [ ]:
ax = sns.lineplot(x=history.epoch, y=history.history['accuracy'])
ax.set(xlabel='epoch', ylabel='accuracy')

Finora abbiamo sempre lavorato sul training set, ma come va il nostro modello su dati su cui non si è mai allenato? Vediamolo tramite la funzione `.evaluate`, che ci restituisce i risultati delle metriche sui dati che forniamo al metodo.

In [ ]:
metrics = model.evaluate(test_X, test_y)
metrics # loss, accuracy

Se vogliamo, possiamo anche farci restituire le predizioni su dati nuovi tramite la funzione `.predict`.

In [ ]:
predictions = model.predict(test_X)
predictions

In maniera equivalente, possiamo chiamare l'oggetto del modello come se fosse una funzione.

In [ ]:
predictions = model(test_X)
predictions

Comunque, con un dataset giocattolo come MNIST possiamo sicuramente fare di meglio, ed è qui che entrano in gioco le __reti neurali deep__.

# Validazione ed Early Stopping

Durante il training delle reti neurali, è fondamentale avere uno strumento che ci permetta di monitorare in che modo si potrebbe comportare il nostro modello se dovesse predire dei dati su cui non si sta allenando durante una generica epoca di training. 

Solitamente, questo strumento viene rappresentato dal __validation set__, su cui facciamo delle predizioni, valutiamo le metriche di quelle predizioni, __ma non ci alleniamo mai__. Questo rappresenta il metodo più comune per valutare la qualità del modello durante la fase di learning.

La prima cosa da fare, è prendere il nostro training set e tirar fuori un pezzetto (solitamente tra il 5% e il 20%) da utilizzare come validation set.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, eval_X, train_y, eval_y = train_test_split(
    train_X, train_y,
    test_size=0.15,
    shuffle=True,
    stratify=train_y
)

In [ ]:
train_X.shape, train_y.shape, eval_X.shape, eval_y.shape

In [ ]:
model = K.Sequential()
model.add(K.layers.Flatten())
model.add(K.layers.Rescaling(scale=1/255.))
#model.add(K.layers.Dense(1000, activation='relu')) # Nuovo layer non lineare
model.add(K.layers.Dense(200, activation='relu')) # Nuovo layer non lineare
model.add(K.layers.Dense(10, activation='softmax'))

model.compile(
    optimizer=K.optimizers.Adam(learning_rate=0.001),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    x=train_X, 
    y=train_y, 
    epochs=100, 
    batch_size=1000, # Questa volta cambiamo il batch size
    validation_data=(eval_X, eval_y)
)

In [ ]:
list(history.history)

In [ ]:
results = pd.DataFrame({
    'epoch': history.epoch,
    'loss': history.history['loss'],
    'val_loss': history.history['val_loss']
})
ax = sns.lineplot(x='epoch', y='value', hue='variable', data=pd.melt(results, ['epoch']))
ax.set(xlabel='epoch', ylabel='loss value')

In [ ]:
results = pd.DataFrame({
    'epoch': history.epoch,
    'loss': history.history['accuracy'],
    'val_loss': history.history['val_accuracy']
})
ax = sns.lineplot(x='epoch', y='value', hue='variable', data=pd.melt(results, ['epoch']))
ax.set(xlabel='epoch', ylabel='accuracy value')

Ora che abbiamo il validation set, possiamo anche utilizzarlo per decidere quando fermare il training, __senza preoccuparci del numero massimo di epoche__.

Keras implementa l'early stopping sotto forma di `callback`. Le callbacks sono funzioni che vengono chiamate in momenti particolari del training, per esempio prima dell'inizio di un'epoca, alla fine di un'epoca, dopo un generico step di learning etc...

In [ ]:
early_stopping = K.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5,
    min_delta=0.0005,
    restore_best_weights=True
)

In [ ]:
model = K.Sequential()
model.add(K.layers.Flatten())
model.add(K.layers.Rescaling(scale=1/255.))
#model.add(K.layers.Dense(1000, activation='relu')) # Nuovo layer non lineare
model.add(K.layers.Dense(200, activation='relu')) # Nuovo layer non lineare
model.add(K.layers.Dense(10, activation='softmax'))

model.compile(
    optimizer=K.optimizers.Adam(learning_rate=0.001),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = model.fit(
    x=train_X, 
    y=train_y, 
    epochs=10000, 
    batch_size=1000,
    validation_data=(eval_X, eval_y),
    callbacks=[early_stopping]
)

In [ ]:
results = pd.DataFrame({
    'epoch': history.epoch,
    'loss': history.history['loss'],
    'val_loss': history.history['val_loss']
})
ax = sns.lineplot(x='epoch', y='value', hue='variable', data=pd.melt(results, ['epoch']))
ax.set(xlabel='epoch', ylabel='loss value')

In [ ]:
results = pd.DataFrame({
    'epoch': history.epoch,
    'loss': history.history['accuracy'],
    'val_loss': history.history['val_accuracy']
})
ax = sns.lineplot(x='epoch', y='value', hue='variable', data=pd.melt(results, ['epoch']))
ax.set(xlabel='epoch', ylabel='accuracy value')

## Salvataggio e caricamento del modello

Ora che abbiamo un modello che potremmo anche voler riutilizzare, avere un metodo per il salvataggio e il caricamento è fondamentale.

La  `model serialization` può essere utile anche quando usate colab, visto che il tempo di utilizzo è limitato e si può disconnettere nel mezzo del processo e potreste voler riprendere il training in seguito.

In [ ]:
model.save('my_model')

In [ ]:
loaded_model = K.models.load_model('my_model')

Ci sono parecchie altre opzioni. Salvataggio in formato H5, salvare solo i pesi, salvare solo l'architettura del modello, etc...  
Guida alla serializzazione dei modelli: https://www.tensorflow.org/guide/keras/save_and_serialize